# **Task 2: Custom CNN**

In [1]:
!pip install pytorch-metric-learning

**Imports and Drive Mount**

In [ ]:
from google.colab import drive
import sys, os

import torch
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms

from pytorch_metric_learning.losses import TripletMarginLoss, ContrastiveLoss
from pytorch_metric_learning.miners import TripletMarginMiner, PairMarginMiner
from pytorch_metric_learning.samplers import MPerClassSampler

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


**Macros**

In [ ]:
# Path to project folder in Drive
project_path = "/content/drive/MyDrive/projectcv2"
sys.path.append(project_path)

# Paths to resources
image_dir = os.path.join(project_path, "data")
class_json = os.path.join(image_dir, "classes.json")
model_path1 = os.path.join(project_path, "resnet50_metric_triplet.pth")
model_path2 = os.path.join(project_path, "resnet50_metric_contrastive.pth")

image_size = 224

device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
from utils import (
    load_dataset,
    get_embedding_model,
    extract_embeddings,
    retrieve_top_k,
    evaluate_retrieval,
    get_embedding_model,
    EmbeddingNet,
    train_or_load_model,
    set_seed
)

set_seed(42)

**Dataset Preparation**

In [ ]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(image_size, scale=(0.8, 1.0)),  # zoom/crop
    transforms.RandomHorizontalFlip(p=0.5),                      # mirror
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # lighting
    transforms.RandomAffine(degrees=15, translate=(0.05, 0.05)),  # slight rotation & shift
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

In [ ]:
dataset = load_dataset(image_dir, class_json, transform)
train_dataset = dataset["reference"]
query_dataset = dataset["query"]

## **Model 1: Triplet Loss**

**Setup and Training**

In [ ]:
model1 = EmbeddingNet().to(device)
loss_fn1 = TripletMarginLoss(margin=0.2)
miner1 = TripletMarginMiner(margin=0.2, type_of_triplets="semihard")
sampler1 = MPerClassSampler(train_dataset.labels, m=4, batch_size=32, length_before_new_iter=len(train_dataset))
optimizer = torch.optim.Adam(model1.parameters(), lr=1e-4)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
model1 = train_or_load_model(
    model=model1,
    model_path=model_path1,
    loss_fn=loss_fn1,
    miner=miner1,
    sampler=sampler1,
    train_dataset=train_dataset,
    device=device,
    epochs=10
)

Training model and saving to /content/drive/MyDrive/projectcv2/resnet50_metric_triplet.pth...
Epoch 1, Loss: 0.0869
Epoch 2, Loss: 0.0729
Epoch 3, Loss: 0.0744
Epoch 4, Loss: 0.0742
Epoch 5, Loss: 0.0373
Epoch 6, Loss: 0.0665
Epoch 7, Loss: 0.0474
Epoch 8, Loss: 0.0463
Epoch 9, Loss: 0.0000
Epoch 10, Loss: 0.0337


**Embedding Extraction**

In [ ]:
query_data = extract_embeddings(model1, query_dataset, device=device)
ref_data = extract_embeddings(model1, train_dataset, device=device)

Extracting embeddings: 100%|██████████| 3/3 [00:01<00:00,  2.12it/s]


**Top-k Retrieval**

In [ ]:
retrieval_result_cos1 = retrieve_top_k(query_data, ref_data, k=1, metric="cosine")
retrieval_result_cos5 = retrieve_top_k(query_data, ref_data, k=5, metric="cosine")
retrieval_result_cos10 = retrieve_top_k(query_data, ref_data, k=10, metric="cosine")

retrieval_result_euc1 = retrieve_top_k(query_data, ref_data, k=1, metric="euclidean")
retrieval_result_euc5 = retrieve_top_k(query_data, ref_data, k=5, metric="euclidean")
retrieval_result_euc10 = retrieve_top_k(query_data, ref_data, k=10, metric="euclidean")

**Evaluation**

In [ ]:
metrics_cos1 = evaluate_retrieval(retrieval_result_cos1, k=1)
metrics_cos5 = evaluate_retrieval(retrieval_result_cos5, k=5)
metrics_cos10 = evaluate_retrieval(retrieval_result_cos10, k=10)

metrics_euc1 = evaluate_retrieval(retrieval_result_euc1, k=1)
metrics_euc5 = evaluate_retrieval(retrieval_result_euc5, k=5)
metrics_euc10 = evaluate_retrieval(retrieval_result_euc10, k=10)

print("Cosine Similarity")
print("Top-1 :", metrics_cos1)
print("Top-5 :", metrics_cos5)
print("Top-10:", metrics_cos10)

print("\nEuclidean Distance")
print("Top-1 :", metrics_euc1)
print("Top-5 :", metrics_euc5)
print("Top-10:", metrics_euc10)

Cosine Similarity
Top-1 : {'Precision@1': 0.95, 'Recall@1': 0.95, 'mAP@1': 0.95}
Top-5 : {'Precision@5': 0.74, 'Recall@5': 0.95, 'mAP@5': 0.9256}
Top-10: {'Precision@10': 0.38, 'Recall@10': 0.95, 'mAP@10': 0.9179}

Euclidean Distance
Top-1 : {'Precision@1': 0.8, 'Recall@1': 0.8, 'mAP@1': 0.8}
Top-5 : {'Precision@5': 0.64, 'Recall@5': 0.9, 'mAP@5': 0.8272}
Top-10: {'Precision@10': 0.355, 'Recall@10': 0.95, 'mAP@10': 0.8248}


## **Model 2: Contrastive Loss**

**Setup and Training**

In [ ]:
model2 = EmbeddingNet().to(device)
loss_fn2 = ContrastiveLoss(pos_margin=0, neg_margin=1)
miner2 = PairMarginMiner(pos_margin=0, neg_margin=1)
sampler2 = MPerClassSampler(train_dataset.labels, m=4, batch_size=32, length_before_new_iter=len(train_dataset))

In [ ]:
model2 = train_or_load_model(
    model=model2,
    model_path=model_path2,
    loss_fn=loss_fn2,
    miner=miner2,
    sampler=sampler2,
    train_dataset=train_dataset,
    device=device,
    epochs=10
)

Training model and saving to /content/drive/MyDrive/projectcv2/resnet50_metric_contrastive.pth...
Epoch 1, Loss: 0.8827
Epoch 2, Loss: 0.8715
Epoch 3, Loss: 0.9495
Epoch 4, Loss: 0.7813
Epoch 5, Loss: 0.6950
Epoch 6, Loss: 0.6646
Epoch 7, Loss: 0.7083
Epoch 8, Loss: 0.6615
Epoch 9, Loss: 0.5920
Epoch 10, Loss: 0.5729


**Embedding Extraction**

In [ ]:
query_data = extract_embeddings(model2, query_dataset, device=device)
ref_data = extract_embeddings(model2, train_dataset, device=device)

Extracting embeddings: 100%|██████████| 3/3 [00:01<00:00,  2.45it/s]


**Top-k Retrieval**

In [ ]:
retrieval_result_cos1 = retrieve_top_k(query_data, ref_data, k=1, metric="cosine")
retrieval_result_cos5 = retrieve_top_k(query_data, ref_data, k=5, metric="cosine")
retrieval_result_cos10 = retrieve_top_k(query_data, ref_data, k=10, metric="cosine")

retrieval_result_euc1 = retrieve_top_k(query_data, ref_data, k=1, metric="euclidean")
retrieval_result_euc5 = retrieve_top_k(query_data, ref_data, k=5, metric="euclidean")
retrieval_result_euc10 = retrieve_top_k(query_data, ref_data, k=10, metric="euclidean")

**Evaluation**

In [ ]:
metrics_cos1 = evaluate_retrieval(retrieval_result_cos1, k=1)
metrics_cos5 = evaluate_retrieval(retrieval_result_cos5, k=5)
metrics_cos10 = evaluate_retrieval(retrieval_result_cos10, k=10)

metrics_euc1 = evaluate_retrieval(retrieval_result_euc1, k=1)
metrics_euc5 = evaluate_retrieval(retrieval_result_euc5, k=5)
metrics_euc10 = evaluate_retrieval(retrieval_result_euc10, k=10)

print("Cosine Similarity")
print("Top-1 :", metrics_cos1)
print("Top-5 :", metrics_cos5)
print("Top-10:", metrics_cos10)

print("\nEuclidean Distance")
print("Top-1 :", metrics_euc1)
print("Top-5 :", metrics_euc5)
print("Top-10:", metrics_euc10)

Cosine Similarity
Top-1 : {'Precision@1': 0.85, 'Recall@1': 0.85, 'mAP@1': 0.85}
Top-5 : {'Precision@5': 0.65, 'Recall@5': 0.9, 'mAP@5': 0.8494}
Top-10: {'Precision@10': 0.35, 'Recall@10': 0.9, 'mAP@10': 0.8262}

Euclidean Distance
Top-1 : {'Precision@1': 0.8, 'Recall@1': 0.8, 'mAP@1': 0.8}
Top-5 : {'Precision@5': 0.64, 'Recall@5': 0.85, 'mAP@5': 0.8061}
Top-10: {'Precision@10': 0.34, 'Recall@10': 0.9, 'mAP@10': 0.818}
